# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f89f4bb2760>
├── 'a' --> tensor([[ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078]])
└── 'x' --> <FastTreeValue 0x7f89f4bb2eb0>
    └── 'c' --> tensor([[-1.6698,  0.6880,  0.9833, -0.6469],
                        [-0.3844, -0.0697, -1.7715,  0.8377],
                        [-0.9949, -0.8132, -1.0637, -0.1916]])

In [4]:
t.a

tensor([[ 1.1360, -1.3562,  0.7570],
        [ 0.1985,  0.3819,  0.4078]])

In [5]:
%timeit t.a

69.8 ns ± 0.108 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f89f4bb2760>
├── 'a' --> tensor([[ 0.8064,  0.7053, -0.0670],
│                   [ 0.6001,  0.2123, -0.5559]])
└── 'x' --> <FastTreeValue 0x7f89f4bb2eb0>
    └── 'c' --> tensor([[-1.6698,  0.6880,  0.9833, -0.6469],
                        [-0.3844, -0.0697, -1.7715,  0.8377],
                        [-0.9949, -0.8132, -1.0637, -0.1916]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.0581 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1360, -1.3562,  0.7570],
               [ 0.1985,  0.3819,  0.4078]]),
    x: Batch(
           c: tensor([[-1.6698,  0.6880,  0.9833, -0.6469],
                      [-0.3844, -0.0697, -1.7715,  0.8377],
                      [-0.9949, -0.8132, -1.0637, -0.1916]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1360, -1.3562,  0.7570],
        [ 0.1985,  0.3819,  0.4078]])

In [11]:
%timeit b.a

58.1 ns ± 0.15 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5689, -0.1487,  0.0797],
               [ 0.0402,  0.8077,  1.2883]]),
    x: Batch(
           c: tensor([[-1.6698,  0.6880,  0.9833, -0.6469],
                      [-0.3844, -0.0697, -1.7715,  0.8377],
                      [-0.9949, -0.8132, -1.0637, -0.1916]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.139 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

833 ns ± 0.671 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 20 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 623 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 579 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f894d1003a0>
├── 'a' --> tensor([[[ 1.1360, -1.3562,  0.7570],
│                    [ 0.1985,  0.3819,  0.4078]],
│           
│                   [[ 1.1360, -1.3562,  0.7570],
│                    [ 0.1985,  0.3819,  0.4078]],
│           
│                   [[ 1.1360, -1.3562,  0.7570],
│                    [ 0.1985,  0.3819,  0.4078]],
│           
│                   [[ 1.1360, -1.3562,  0.7570],
│                    [ 0.1985,  0.3819,  0.4078]],
│           
│                   [[ 1.1360, -1.3562,  0.7570],
│                    [ 0.1985,  0.3819,  0.4078]],
│           
│                   [[ 1.1360, -1.3562,  0.7570],
│                    [ 0.1985,  0.3819,  0.4078]],
│           
│                   [[ 1.1360, -1.3562,  0.7570],
│                    [ 0.1985,  0.3819,  0.4078]],
│           
│                   [[ 1.1360, -1.3562,  0.7570],
│                    [ 0.1985,  0.3819,  0.4078]]])
└── 'x' --> <FastTreeValue 0x7f894d1007f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 70.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f89f4ba7550>
├── 'a' --> tensor([[ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078],
│                   [ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078],
│                   [ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078],
│                   [ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078],
│                   [ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078],
│                   [ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078],
│                   [ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078],
│                   [ 1.1360, -1.3562,  0.7570],
│                   [ 0.1985,  0.3819,  0.4078]])
└── 'x' --> <FastTreeValue 0x7f89f4ba77c0>
    └── 'c' --> tensor([[-1.6698,  0.6880,  0.9833, -0.6469],
                        [-0.3844, -0.0697, -1.7715,  0.8377],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 72.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

54.8 µs ± 60.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.6698,  0.6880,  0.9833, -0.6469],
                       [-0.3844, -0.0697, -1.7715,  0.8377],
                       [-0.9949, -0.8132, -1.0637, -0.1916]],
              
                      [[-1.6698,  0.6880,  0.9833, -0.6469],
                       [-0.3844, -0.0697, -1.7715,  0.8377],
                       [-0.9949, -0.8132, -1.0637, -0.1916]],
              
                      [[-1.6698,  0.6880,  0.9833, -0.6469],
                       [-0.3844, -0.0697, -1.7715,  0.8377],
                       [-0.9949, -0.8132, -1.0637, -0.1916]],
              
                      [[-1.6698,  0.6880,  0.9833, -0.6469],
                       [-0.3844, -0.0697, -1.7715,  0.8377],
                       [-0.9949, -0.8132, -1.0637, -0.1916]],
              
                      [[-1.6698,  0.6880,  0.9833, -0.6469],
                       [-0.3844, -0.0697, -1.7715,  0.8377],
                       [-0.9949, -0.8132, -1.0637, -0.1916]],

In [26]:
%timeit Batch.stack(batches)

77 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.6698,  0.6880,  0.9833, -0.6469],
                      [-0.3844, -0.0697, -1.7715,  0.8377],
                      [-0.9949, -0.8132, -1.0637, -0.1916],
                      [-1.6698,  0.6880,  0.9833, -0.6469],
                      [-0.3844, -0.0697, -1.7715,  0.8377],
                      [-0.9949, -0.8132, -1.0637, -0.1916],
                      [-1.6698,  0.6880,  0.9833, -0.6469],
                      [-0.3844, -0.0697, -1.7715,  0.8377],
                      [-0.9949, -0.8132, -1.0637, -0.1916],
                      [-1.6698,  0.6880,  0.9833, -0.6469],
                      [-0.3844, -0.0697, -1.7715,  0.8377],
                      [-0.9949, -0.8132, -1.0637, -0.1916],
                      [-1.6698,  0.6880,  0.9833, -0.6469],
                      [-0.3844, -0.0697, -1.7715,  0.8377],
                      [-0.9949, -0.8132, -1.0637, -0.1916],
                      [-1.6698,  0.6880,  0.9833, -0.6469],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
